In [9]:
import numpy as np
from scipy.special import softmax

# softmax_logits = softmax(logits/T, axis=1)
labels = np.asarray([0, 0])
n = len(labels)
softmax_logits = np.asarray([[.25, .05, .6, .1], [.25, .05, .6, .1]])
y_true_scores = softmax_logits[np.arange(n), labels]
mask = softmax_logits >= np.expand_dims(y_true_scores, 1)
(mask * softmax_logits).sum(1)



array([0.85, 0.85])

In [15]:
from evaluate import load_pickle, split_data

n_calib = 1000
batch_size = 128
num_worker = 4
alpha = 0.1
ts = True
seed = None
file_name = '/home/royhirsch/conformal/data/embeds_n_logits/imnet1k_r18/valid.pickle'

data = load_pickle(file_name)
valid_data, calib_data = split_data(data, n_calib, seed=0)

logits = calib_data['preds']
labels = calib_data['labels']
n = len(labels)


In [18]:
softmax_logits = softmax(logits / 1, axis=1)
y_true_scores = softmax_logits[np.arange(n), labels]
mask = softmax_logits >= np.expand_dims(y_true_scores, 1)
scores = (mask * softmax_logits).sum(1)

q_level = np.ceil((n+1)*(1-alpha))/n
qhat = np.quantile(scores, q_level, interpolation="higher")
print(qhat)

0.99940026


In [20]:
val_logits = valid_data['preds'][:10000,:]
val_labels = valid_data['labels'][:10000]

scores = softmax(val_logits / 1, axis=1)

asending_inds = scores.argsort(1)[:, ::-1]
asending_scores = np.sort(scores, axis=1)[:, ::-1]
cumsum_asending_scores = asending_scores.cumsum(axis=1)

sets = np.zeros_like(scores)
for i, row in enumerate(cumsum_asending_scores):
    indices = np.where(row >= qhat)[0]
    if len(indices) > 0:
        sets[i, asending_inds[i, :indices[0]]] = 1
    else:
        # empty sets
        continue
sets.sum(1).mean()

189.7559

In [26]:
sets.sum(1)[:12]

array([898.,   4.,  28., 120.,  59.,   2., 421., 645., 334.,  37.,  24.,
       818.], dtype=float32)